# Implanatação de algumas Bibliotacas

In [ ]:
import csv
import numpy as np
import pandas as pd
import sys
from tkinter import Tk
from tkinter import filedialog
from datetime import date, datetime, timedelta
#import os
#from ciso8601 import parse_datetime

# Criando variaveis para os arquivos permitidos

In [ ]:
#p_cmpid   = pd.read_excel('PQ376.xlsx')      ### leitura do arquivo com os sinais dos cpmids do quadro 376
#p_tpmoid  = pd.read_excel('Pt376.xlsx')      ### leitura do arquivo com os sinais dos tpmoids do quadro 376
#param     = pd.read_excel('LQ376.xlsx')      ### leitura do arquivo com o layout atualizado do quadro 376

p_cmpid2  = pd.read_excel('PQ377.xlsx')      ### leitura do arquivo com os sinais dos cpmids do quadro 377
param2    = pd.read_excel('LQ377.xlsx')      ### leitura do arquivo com o layout atualizado do quadro 377

In [ ]:
control  = 0                                              ### controle de backup    
#p_cmpid['cmpid'] = p_cmpid.cmpid.astype(str)              ### transforma o campo cmpid - QE 376 - do arq auxiliar para string 
p_cmpid2['cmpid'] = p_cmpid2.cmpid.astype(str)            ### transforma o campo cmpid - QE 377 - do arq auxiliar para string 

In [ ]:
## FUNÇÃO PARA SELEÇÃO DO ARQUIVO

class qe_s:

    def __init__(self):
        self.df = {nome[s]: [] for s in range(nome.shape[0])}
  
    def make_df(self):
        lines = []
        root = Tk()
        root.attributes("-topmost", True)
        root.withdraw()
        for file in filedialog.askopenfilenames(filetypes=[("Arquivos de Text", "*.txt")]):
            for line in open(file, encoding="utf-8"):
                line = line.replace(",",".").strip()
                for x in range(nome.shape[0]):
                    self.df[nome[x]].append(line[i[x]:j[x]])
                    
        return pd.DataFrame(self.df)

In [ ]:
nome     = param2.Campo                      ### guarda os nomes das colunas existentes no quadro 376
i        = param2.Ind_inf                    ### guarda a posição no arquivo que inicia um determinado campo
j        = param2.Ind_sup                    ### guarda a posição no arquivo que termina um determinado campo

quadro = qe_s()       
df_main = quadro.make_df()
df_main

## SELECINE O(S) ARQUIVO(S) DESEJADO(S)

In [ ]:
###################################
###Importacao do Quadro377.xlsx ###
####que serão usados para fins#####
######### de comparação ###########
###################################

#Pasta CMPID
arq_cmpid = pd.read_excel('Quadro377.xlsx','CMPID',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta ENTCODIGO
arq_entcod = pd.read_excel('Quadro377.xlsx','Cod_SUSEP',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta MRFMESANO
arq_mrfmesano = pd.read_excel('Quadro377.xlsx','MRFMESANO',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta RAMCODIGO
arq_ramcod = pd.read_excel('Quadro377.xlsx','RAMCODIGO',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta Bib_DefCamposEstatísticos
arq_defcamesta = pd.read_excel('Quadro377.xlsx','Bib_DefCamposEstatísticos',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

In [ ]:
# Criando uma lista para guardar todas as criticas impeditivas
criticas_impeditivas = []
criticas = []

# Criticas (Validacoes)

In [ ]:
#7393.3 Verifica se o campo sequencial ESLSEQ é uma sequência válida, que se inicia em 0000001
array3 = df_main['ESLSEQ']
x = list(array3)                                     ##Dados do arquivo
arrayteste = x[:]
arrayteste.sort()                                    ##Dados sequenciados
if (arrayteste == x):                                ##Comparando
    print('Tudo certo, esta de forma sequencial.')
else: 
    print('ERRO!!!, Não prossiga.')
    critica7393_3 = str('Critica 7393.3, Não temos uma sequencia valida na coluna ESRSEQ')
    criticas_impeditivas.append(critica7393_3)

In [ ]:
#7393.2 Verifica o tamanho padrão da linha (deve conter 101 caracteres)
tamanho = 0
for i in range(0,df_main.shape[0]):                    #Passando por cada linha
    for iten in df_main.columns:                       #Passando por cada coluna
        tamanho = tamanho + len(df_main[iten][i])      #Contando Caracteres por linha
    if tamanho != 101:
        print('Erro na linha ',i,' o número de caracteres é de: ',tamanho)
        critica7393_2 = ("Critica 7393.2, Erro na linha ",i," o número de caracteres é de: ",tamanho)
        criticas_impeditivas.append(critica7393_2)
    tamanho = 0

In [ ]:
#7393.4 Verifica se o campo ENTCODIGO corresponde à sociedade que está enviando o FIP/SUSEP
entcod = set(arq_entcod['Cod_SUSEP'].astype('str'))                   #Escolhendo a coluna do arquivo modelo
dadosentcod = set(df_main['ENTCODIGO'])                               #Escolhendo a coluna dos dados
for itemtentcod in dadosentcod: 
    if itemtentcod not in entcod:
        print('ERRO!! ENTCODIGO não corresponde a uma operação valida', itemtentcod)
        critica7393_4 = ('Critica 7393.4, ENTCODIGO não corresponde a uma operação valida', itemtentcod)
        criticas.append(critica7393_4)
    else:
        print('ENTCODIGO corresponde a um tipo de operação válida.', itemtentcod)
        continue

In [ ]:
#7393.5 Verifica se o campo MRFMESANO corresponde, respectivamente, ao ano, mês e último dia do mês de referência do FIP/SUSEP
df_main.loc[:,'DTMRFMESANO'] = df_main['MRFMESANO'].astype('datetime64')     #Criando uma nova coluna no formato Date
mrfmesano = set(arq_mrfmesano['DATAS'].astype('datetime64'))                 #Escolhendo a coluna do arquivo modelo
dadosmrfmesano = set(df_main['DTMRFMESANO'])                                 #Escolhendo a coluna dos dados
for itemmrfmesano in dadosmrfmesano: 
    if itemmrfmesano not in mrfmesano:
        print('ERRO!! MRFMESANO não corresponde a uma operação valida', itemmrfmesano)
        critica7393_5 = ('Critica 7393.5, MRFMESANO não corresponde a uma Data valida', itemmrfmesano)
        criticas.append(critica7393_5)
    else:
        print('MRFMESANO corresponde a uma Data válida.', itemmrfmesano)
        continue

In [ ]:
#7393.6 Verifica se o campo QUAID corresponde ao quadro 377
array6 = set(df_main['QUAID'])           ##Verificando todos os Quadros que estamos tratando no documento.
for itemarray6 in array6:
    if itemarray6 == '377':
        print('Este se trata do Quadro:',itemarray6)
    else:
        print('ERRO!! Este se trata do Quadro:',itemarray6)
        critica7393_6 = ('Critica 7393.6, Este esta se tratando do(s) Quadro(s):',itemarray6)
        criticas.append(critica7393_6)

In [ ]:
#7393.7 Verifica se o campo CMPID corresponde a um tipo de operação válida (conforme tabela Bib_DefCamposEstatísticos)
defcamesta = set(arq_defcamesta['CMPID'].astype('str'))             #Escolhendo a coluna do arquivo modelo
dadoscmpid = set(df_main['CMPID'])                                 #Escolhendo a coluna dos dados
for itemcmpid in dadoscmpid: 
    if itemcmpid not in defcamesta:
        print('ERRO!! O campo CMPID não corresponde a um tipo de operação válida (conforme tabela Bib_DefCamposEstatísticos).', itemcmpid)
        critica7393_7 = ('Critica 7393.7, O campo CMPID não corresponde a um tipo de operação válida (conforme tabela Bib_DefCamposEstatísticos).', itemcmpid)
        criticas.append(critica7393_7)
    else:
        print('O campo CMPID corresponde a um tipo de operação válida (conforme tabela Bib_DefCamposEstatísticos).', itemcmpid)
        continue

In [ ]:
#7393.10 Verifica se o campo RAMCODIGO corresponde, respectivamente, a um grupo de ramos e ramo válidos e operados pela companhia no mês de referência
ramcod = set(arq_ramcod['ramo_s1'].astype('str'))                       #Escolhendo a coluna do arquivo
dadosramcod = set(df_main['RAMCODIGO'])                                 #Escolhendo a coluna dos dados
for itemramcod in dadosramcod: 
    if itemramcod not in ramcod:
        print('ERRO!! RAMCODIGO não corresponde a uma operação valida', itemramcod)
        critica7393_10 = ('Critica 7393.10, RAMCODIGO não corresponde a uma operação valida', itemramcod)
        criticas.append(critica7393_10)
    else:
        print('RAMCODIGO corresponde a um tipo de operação válida.', itemramcod)
        continue

In [ ]:
#7393.11 Verifica se o campo RAMCODIGO não foi preenchido para os ramos 0588, 0589, 0983, 0986, 0991, 0992, 0994, 0996, 1066, 1383, 1386, 1391, 1392, 1396, 1603 e 2201
ramcodver = set(arq_ramcod['RAMCODIGO_não_fazem_parte_376'].astype('str'))       #Escolhendo a coluna do arquivo modelo
dadosramcod = set(df_main['RAMCODIGO'])                                 #Escolhendo a coluna dos dados
for itemramcod in dadosramcod: 
    if itemramcod in ramcodver:
        print('ERRO!! RAMCODIGO não corresponde a uma operação valida', itemramcod)
        critica7393_11 = ('Critica 7393.11, RAMCODIGO não corresponde a uma operação valida', itemramcod)
        criticas.append(critica7393_11)
    else:
        print('RAMCODIGO corresponde a um tipo de operação válida.', itemramcod)
        continue

In [ ]:
#7393.13 Verifica se os campos ESLDATAINICIO, ESLDATAFIM, ESLDATAOCORR e ESLDATAREG correspondem a uma data válida

#####################################################
##Readicionando as colunas de Data de str para date##
#####################################################

df_main.loc[:,'DTESLDATAINICIO'] = df_main['ESLDATAINICIO'].astype('datetime64')
df_main.loc[:,'DTESLDATAFIM'] = df_main['ESLDATAFIM'].astype('datetime64')
df_main.loc[:,'DTESLDATAOCORR'] = df_main['ESLDATAOCORR'].astype('datetime64')
df_main.loc[:,'DTESLDATAREG'] = df_main['ESLDATAREG'].astype('datetime64')

#Adicionado colunas DTESRDATAINICIO, DTESRDATAFIM, DTESRDATAOCORR, DTESRDATAREG e DTESRDATACOMUNICA no tipo Date

In [ ]:
#7393.12 Verifica se o valor dos campos ESLVALORMOV é float

###########################################
##Readicionando as colunas str para float##
###########################################

df_main.loc[:,'fESRVALORMOV'] = df_main['ESLVALORMOV'].astype('float64')

#Adicionado coluna ESLVALORMOV no tipo Float

In [ ]:
criticas_impeditivas  #Lista das criticas impeditivas

In [ ]:
df_criticas_i = pd.DataFrame(criticas_impeditivas)     #criacao do DataFrame das criticas impeditivas
df_criticas_i = df_criticas_i.rename(columns={0:'Descrição'})
df_criticas_i

In [ ]:
if criticas_impeditivas == []:                             # Verificando se a lista de criticas esta vazia
    print('Nenhuma Critica Impeditiva encontrada no Quadro 377')
else:
    print('Foi encontrada uma ou mais Criticas Impeditivas no Quadro 377')
    df_criticas_i.to_csv('Criticas_Impeditivas_377.csv')   # Gerando csv das criticas impeditivas

# Criticas (Validacoes) Não Impeditivas

In [ ]:
# Criando DataFrame copia com Campos CMPID, ESLCODCESS, ENTCODIGO no formato int
df_mains = df_main.copy()
df_mains['CMPID'] = df_mains['CMPID'].astype('int64')
df_mains['ESLCODCESS'] = df_mains['ESLCODCESS'].astype('int64')
df_mains['ENTCODIGO'] = df_mains['ENTCODIGO'].astype('int64')

In [ ]:
#7393.14 Valida a correspondência entre os campos CMPID e ESLCODCESS
relacao15_15 = df_mains[(df_mains['CMPID'] == 1015) & (df_mains['ESLCODCESS'].values != df_mains['ENTCODIGO'].values)]
relacao15_20 = df_mains[(df_mains['CMPID'] == 1020) & (df_mains['ESLCODCESS'].values != df_mains['ENTCODIGO'].values)]
relacao15_25 = df_mains[(df_mains['CMPID'] == 1025) & (df_mains['ESLCODCESS'].values != df_mains['ENTCODIGO'].values)]

In [ ]:
i = 19   #Definindo numero de colunas

# Adicionando criticas com CMPIDs 1015, 1020, 1025 na lista 'criticas'
if relacao15_15.shape != (0,i): 
    critica7392_15_15 = ('Critica 7392.15_15, O CMPID é 1015 e o ESLCODCESS é diferente do ENTCODIGO') 
    criticas.append(critica7392_15_15)
elif relacao15_20.shape != (0,i): 
    critica7392_15_20 = ('Critica 7392.15_20, O CMPID é 1020 e o ESLCODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_15_20)
elif relacao15_25.shape != (0,i): 
    critica7392_15_25 = ('Critica 7392.15_25, O CMPID é 1025 e o ESLCODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_15_25)
else: print('CMPIDs 1015, 1020, 1025. Valida campo ESLCODCESS')

In [ ]:
#7393.14 Valida a correspondência entre os campos CMPID e ESLCODCESS
relacao15_17 = df_mains[(df_mains['CMPID'] == 1017) & (df_mains['ESLCODCESS'].values <= 1) & (df_mains['ESLCODCESS'].values >= 9999)] 
relacao15_21 = df_mains[(df_mains['CMPID'] == 1021) & (df_mains['ESLCODCESS'].values <= 1) & (df_mains['ESLCODCESS'].values >= 9999)] 
relacao15_22 = df_mains[(df_mains['CMPID'] == 1022) & (df_mains['ESLCODCESS'].values <= 1) & (df_mains['ESLCODCESS'].values >= 9999)] 

In [ ]:
# Adicionando criticas com CMPIDs 1017, 1021, 1022 na lista 'criticas'
if relacao15_17.shape != (0,i): 
    critica7392_15_17 = ('Critica 7392.15_17, O CMPID é 1017 e o ESLCODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_15_17)
elif relacao15_21.shape != (0,i): 
    critica7392_15_21 = ('Critica 7392.15_21, O CMPID é 1021 e o ESLCODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_15_21)
elif relacao15_22.shape != (0,i): 
    critica7392_15_22 = ('Critica 7392.15_22, O CMPID é 1022 e o ESLCODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_15_22)
else: print('CMPIDs 1017, 1021, 1022. Valida campo ESLCODCESS')

In [ ]:
#7393.14 Valida a correspondência entre os campos CMPID e ESLCODCESS
relacao15_18 = df_mains[(df_mains['CMPID'] == 1018) & (df_mains['ESLCODCESS'].values <= 30000) & (df_mains['ESLCODCESS'].values >= 59999)] 
relacao15_19 = df_mains[(df_mains['CMPID'] == 1019) & (df_mains['ESLCODCESS'].values <= 30000) & (df_mains['ESLCODCESS'].values >= 59999)] 
relacao15_23 = df_mains[(df_mains['CMPID'] == 1023) & (df_mains['ESLCODCESS'].values <= 30000) & (df_mains['ESLCODCESS'].values >= 59999)] 
relacao15_24 = df_mains[(df_mains['CMPID'] == 1024) & (df_mains['ESLCODCESS'].values <= 30000) & (df_mains['ESLCODCESS'].values >= 59999)]  

In [ ]:
# Adicionando criticas com CMPIDs 1018, 1019, 1023, 1024 na lista 'criticas'
if relacao15_18.shape != (0,i): 
    critica7392_15_18 = ('Critica 7392.15_18, O CMPID é 1018 e o ESLCODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_15_18)
elif relacao15_19.shape != (0,i): 
    critica7392_15_19 = ('Critica 7392.15_19, O CMPID é 1019 e o ESLCODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_15_19)
elif relacao15_23.shape != (0,i): 
    critica7392_15_23 = ('Critica 7392.15_23, O CMPID é 1023 e o ESLCODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_15_23)
elif relacao15_24.shape != (0,i): 
    critica7392_15_24 = ('Critica 7392.15_24, O CMPID é 1024 e o ESLCODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_15_24)
else: print('CMPIDs 1018, 1019, 1023, 1024. Valida campo ESLCODCESS')

In [ ]:
#7393.15 Verifica se o campo ESLCODCESS corresponde a um código de sociedade válido
dadosesrcodcess = set(df_main['ESLCODCESS'])       #Escolhendo a coluna dos dados
for itemtesrcodcess in dadosesrcodcess: 
    if itemtesrcodcess not in entcod:
        print('ERRO!! ESLCODCESS não corresponde a um código de sociedade válido', itemtesrcodcess)
        critica7393_15 = ('Critica 7393.15, ESLCODCESS corresponde a um código de sociedade válido', itemtesrcodcess)
        criticas.append(critica7393_15)
    else:
        print('ENTCODIGO corresponde a um tipo de operação válida.', itemtesrcodcess)
        continue

In [ ]:
#7393.16 Se o tipo de operação for 'direto - administrativo' ou 
#'cosseguro aceito - administrativo' ou 'cosseguro cedido - administrativo' ou 
#'recuperação de sinistros não pagos - administrativo' ou 
#'recuperação de sinistros já pagos - administrativo', 
#verifica se o ano da data dos campos ESLDATAINICIO, ESLDATAFIM, ESLDATAOCORR 
#e ESLDATAREG está entre os limites de trinta anos para mais ou para menos do 
#ano da data do campo MRFMESANO
df_dates_comparacao = pd.DataFrame(df_main['DTMRFMESANO'] - timedelta(days = 10950))
df_dates_comparacao.loc[:,'DTMRFMESANO_max'] = df_main['DTMRFMESANO'] + timedelta(days = 10950)

In [ ]:
mask16_i = (df_main['DTESLDATAINICIO'] < df_dates_comparacao['DTMRFMESANO']) | (df_main['DTESLDATAINICIO'] > df_dates_comparacao['DTMRFMESANO_max'])
df16_i = df_main.loc[mask16_i] 

if len(df16_i) > 0:
    df16_i.loc[:,'ID da Critica'] = ('CRITICA 7394.16')

df16_i

In [ ]:
mask16_f = (df_main['DTESLDATAFIM'] < df_dates_comparacao['DTMRFMESANO']) | (df_main['DTESLDATAFIM'] > df_dates_comparacao['DTMRFMESANO_max'])
df16_f = df_main.loc[mask16_f] 

if len(df16_f) > 0:
    df16_f.loc[:,'ID da Critica'] = ('CRITICA 7394.16')
df16_f

In [ ]:
mask16_o = (df_main['DTESLDATAOCORR'] < df_dates_comparacao['DTMRFMESANO']) | (df_main['DTESLDATAOCORR'] > df_dates_comparacao['DTMRFMESANO_max'])
df16_o = df_main.loc[mask16_o]

if len(df16_o) > 0:
    df16_o.loc[:,'ID da Critica'] = ('CRITICA 7394.16')
df16_o

In [ ]:
mask16_r = (df_main['DTESLDATAREG'] < df_dates_comparacao['DTMRFMESANO']) | (df_main['DTESLDATAREG'] > df_dates_comparacao['DTMRFMESANO_max'])
df16_r = df_main.loc[mask16_r] 

if len(df16_r) > 0:
    df16_r.loc[:,'ID da Critica'] = ('CRITICA 7394.16')
df16_r

In [ ]:
#7393.17 Verifica se a data do campo ESLDATAFIM é posterior à data do campo ESLDATAINICIO 
mask17 = (df_main['DTESLDATAFIM'] < df_main['DTESLDATAINICIO']) #Criando uma mascara para realizar a validação

df17 = df_main.loc[mask17]                                      #Transformando a mascara em DataFrame

if len(df17) > 0:
    df17.loc[:,'ID da Critica'] = ('CRITICA 7394.17')
df17                                                            #Apresentando o DataFrame

In [ ]:
#7393.18 Verifica se a data do campo ESLDATAOCORR está entre as datas dos campos ESLDATAINICIO e ESLDATAFIM 
mask18 = (df_main['DTESLDATAOCORR'] < df_main['DTESLDATAINICIO']) | (df_main['DTESLDATAOCORR'] > df_main['DTESLDATAFIM'])

df18 = df_main.loc[mask18]

if len(df18) > 0:
    df18.loc[:,'ID da Critica'] = ('CRITICA 7394.18')
df18

In [ ]:
#7393.19 Verifica se a data do campo ESLDATAOCORR é igual ou anterior à data do campo ESLDATAREG
mask19 = (df_main['DTESLDATAOCORR'] > df_main['DTESLDATAREG'])
df19 = df_main.loc[mask19]

if len(df19) > 0:
    df19.loc[:,'ID da Critica'] = ('CRITICA 7394.19')
df19

In [ ]:
#7393.20 Verifica se a data dos campos ESLDATAINICIO, ESLDATAOCORR e ESLDATAREG é igual ou anterior à data do campo MRFMESANO
mask20 = (df_main['DTMRFMESANO'] < df_main['DTESLDATAINICIO']) | (df_main['DTMRFMESANO'] < df_main['DTESLDATAOCORR']) | (df_main['DTMRFMESANO'] < df_main['DTESLDATAREG'])
df20 = df_main.loc[mask20]

if len(df20) > 0:
    df20.loc[:,'ID da Critica'] = ('CRITICA 7394.20')
df20

In [ ]:
#7393.21 Para cada ramo, compara os valores registrados no campo ESLVALORMOV com os valores informados no Quadro 07 do FIP/SUSEP (tolerância: R$ 1.000,00 ou 1% do valor informado no respectivo campo do Quadro 07, o que for maior)

In [ ]:
#7393.22 Verifica se o valor do campo ESLVALORMOV é maior do que zero
mask22 = (df_main['fESRVALORMOV'] <= 0.0)
df22 = df_main.loc[mask22]

if len(df22) > 0:
    df22.loc[:,'ID da Critica'] = ('CRITICA 7394.22')
df22

In [ ]:
#7393.23 Verifica se o campo ESLNUMSIN é diferente de 0.
mask23 = (df_main['ESLNUMSIN'] == 0.0)
df23 = df_main.loc[mask23]

if len(df23) > 0:
    df23.loc[:,'ID da Critica'] = ('CRITICA 7394.23')
df23

In [ ]:
################################################
# Adicionando Nome do Criador do output e data #
################################################

df_main.loc[:,'Criador'] = input("Insira seu nome: ") ### insere a coluna com o nome do criador da tabela
data_presente = datetime.now()
df_main.loc[:,'Data do output'] = data_presente ### insere a coluna com a data de criacao da tabela

In [ ]:
df_main

In [ ]:
criticas

In [ ]:
df_criticas = pd.DataFrame(criticas)
df_criticas = df_criticas.rename(columns={0:'Descrição'})
df_criticas

In [ ]:
df_criticas_2 = pd.concat([df16_i,df16_f,df16_o,df16_r,df17,df18,df19,df20,df22,df23])
df_criticas_2

In [ ]:
if criticas == []:                   # Verificando se a lista de criticas esta vazia
    print('Nenhuma Critica NÃO Impeditiva encontrada no Quadro 377')
else:
    print('Foi encontrada uma ou mais Criticas NÃO Impeditivas no Quadro 377')
    df_criticas.to_csv('Criticas_nao_impeditivas377.csv')     # Gerando csv das criticas impeditivas

In [ ]:
if len(df_criticas_2) == 0:                             # Verificando se o DataFrame de criticas esta vazia
    print('Nenhuma Critica NÃO Impeditiva encontrada no Quadro 377')
else:
    print('Foi encontrada uma ou mais Criticas NÃO Impeditivas no Quadro 377')
    df_criticas_2.to_csv('Criticas_nao_impeditivas377_2.csv')         # Gerando csv das criticas impeditivas

In [ ]:
df_main.to_csv('377_Trabalho.csv')